The following is from [this article](https://medium.com/towards-data-science/monitoring-machine-learning-models-in-production-why-and-how-13d07a5ff0c6) in Medium.

# #1 Monitor the model performance metrics

To detect any sign of model degradation, one of the direct and effective ways is to keep track of the performance metrics over time.

- [Performance metrics for regression model](https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b): R-squared, Root mean squared error (RMSE), and mean absolute error (MAE)
- [Performance metrics for classification model](https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226): Precision, recall, and F1-score

These performance metrics collected from the initial deployment serve as benchmarks for ongoing monitoring and evaluation. Periodically reassessing them is crucial whenever a new batch of ground truth data is collected, such as upon completion of a marketing campaign. If the error metrics rise above a predefined threshold, or if the metrics such as R-squared fall below the threshold, it is necessary to consider re-executing the data engineering process and retraining the model.

While this monitoring approach provides valuable insights into any drifts, it tends to lag. We can take a more proactive approach to monitor the latest input data.

# #2 Detect the changes in data distribution

Instead of waiting for sufficient input data for reliable evaluation of model performance, we can apply statistical methods for comparing the data distributions of two datasets. In our case, we can determine if the distribution of the training dataset is the same as that of the latest production dataset. If it is not statistically confident that two distributions are the same, it suggests a drift in the model. This is as a proxy for performance changes.

- Kolmogorov-Smirnov Test ([K-S Test](https://towardsdatascience.com/kolmogorov-smirnov-test-84c92fb4158d)): nonparametric test (i.e. with no assumption on the underlying data distribution) **for numeric features**, it is more sensitive near the center of the distribution than at the tails.

> Interpretation: When p-value < 0.05, an alert indicating the presence of a drift is triggered.

- Population Stability Index ([PSI](https://towardsdatascience.com/psi-and-csi-top-2-model-monitoring-metrics-924a2540bed8)): A statistical test that can be applied to **both numeric and categorical variables**. It is a metric that shows how each variable has diverged independently from the baseline values. It is sometimes called the Characteristic Stability Index (CSI) when evaluating the distribution of features rather than target variables.

> Interpretation: A value 0~0.1 means no significant distribution change; a value 0.1~0.2 considered a moderate distribution change; and a value larger than 0.2 interpreted as a significant distribution change.

Other famous statistical tests include [Kullback-Leibler divergence](https://towardsdatascience.com/understanding-kl-divergence-f3ddc8dff254), [Jensen-Shannon divergence](https://towardsdatascience.com/how-to-understand-and-use-jensen-shannon-divergence-b10e11b03fd6), and [Wasserstein distance](https://towardsdatascience.com/the-gromov-wasserstein-distance-835c39d4751d).

# #3 Monitor drift using a sliding window approach

Any delay in detecting data or pattern drifts results in a time gap, potentially leading to discrepancies between the ground truth and model predictions. To bridge the gap, are there any further advancements available? One promising idea is leveraging streaming data instead of batch data.

The Adaptive Windowing ([ADWIN](https://scikit-multiflow.readthedocs.io/en/stable/api/generated/skmultiflow.drift_detection.ADWIN.html)) algorithm utilizes a sliding window approach to effectively detect concept drift. Unlike traditional fixed-size windows, ADWIN decides the size of the window by cutting the statistics window at different points. Whenever newly arriving data comes, ADWIN analyses the statistics and identifies the point at which two sub-windows demonstrate notable differences in their means.

> Interpretation: When the absolute difference between the two means exceeds a pre-defined threshold, an alert indicating the presence of a drift is triggered.

# Example Walkthrough

We will utilize a [dataset](https://www.kaggle.com/datasets/parisrohan/credit-score-classification?datasetId=2289007&sortBy=voteCount&select=train.csv) obtained from Kaggle. The dataset comprises 100k records, encompassing 28 features that describe the customer demographics and their credit-related history.

Our objective is to segregate customers in a global financial company into credit score brackets. The target variable is Credit_Score, a categorical measure classified as ‘Poor’, ‘Standard’, and ‘Good’.

Examples of features include:

- Occupation: Occupation of the customers (e.g. scientist, teacher, engineer, etc.)
- Annual_Income: Annual income of the customers
- Credit_History_Age: The age of customers’ credit history
- Payment_Behaviour: 6 groups of payment behaviors based on spending frequency (low/ high) and payment amount (small/ medium/ large)

# Model performance metrics

Let’s start by applying data cleansing and transformation techniques, including but not limited to:

- Correcting/Removing records with missing values or incorrect data (e.g. age < 0), or duplicates
- Detecting and handling data outliers
- Performing min-max scaling on numeric variables
- Applying label encoding on categorical variables

*This part requires in-depth exploratory data analysis. However, since the focus is on sharing the monitoring strategies, the transformations performed are not discussed in detail here.*

Afterward, we split the dataset and developed the advanced gradient boosting algorithm, LightGBM.

In [1]:
import warnings

import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("ml_monitoring_example/train.csv", low_memory=False)

In [3]:
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


The following codes for data preprocessing is from [Kaggle](https://www.kaggle.com/code/supawitongkariyapong/project-credit-score-classification)

In [4]:
# Drop the column which is out of model scope
d_col = [
    "ID",
    "Customer_ID",
    "Month",
    "Name",
    "SSN",
    "Monthly_Inhand_Salary",
    "Num_Bank_Accounts",
    "Num_Credit_Card",
    "Interest_Rate",
    "Num_of_Loan",
    "Type_of_Loan",
    "Changed_Credit_Limit",
    "Num_Credit_Inquiries",
    "Credit_Mix",
    "Credit_Utilization_Ratio",
    "Amount_invested_monthly",
    "Occupation",
]
df = df.drop(d_col, axis=1)

In [5]:
# Drop columns with nan
df = df.dropna()

In [6]:
# Drop the incorrect data
# df = df[df["Occupation"].str.contains("_______") == False]
df = df[df["Payment_Behaviour"].str.contains("!@9#%8") == False]

In [7]:
# Revise the incorrect data whole table
sym = "\\`*_{}[]()>#@+!$:;"
col_int = [
    "Age",
    "Delay_from_due_date",
    "Num_of_Delayed_Payment",
    "Outstanding_Debt",
    "Total_EMI_per_month",
    "Monthly_Balance",
    "Annual_Income",
]
col_str = ["Credit_History_Age", "Payment_of_Min_Amount", "Credit_Score"]
for i in col_int:
    for c in sym:
        df[i] = df[i].astype(str).str.replace(c, "")
for i in col_str:
    for c in sym:
        df[i] = df[i].replace(c, "")

In [8]:
# Transform the information to the value
df["Credit_History_Age"] = (
    df["Credit_History_Age"].astype(str).str.replace(" Years and ", ".")
)
df["Credit_History_Age"] = (
    df["Credit_History_Age"].astype(str).str.replace("Months", "")
)

In [9]:
# Transform the information to the value as level
df["Payment_Behaviour"] = (
    df["Payment_Behaviour"]
    .astype(str)
    .str.replace("Low_spent_Small_value_payments", "1")
)
df["Payment_Behaviour"] = (
    df["Payment_Behaviour"]
    .astype(str)
    .str.replace("Low_spent_Medium_value_payments", "2")
)
df["Payment_Behaviour"] = (
    df["Payment_Behaviour"]
    .astype(str)
    .str.replace("Low_spent_Large_value_payments", "3")
)
df["Payment_Behaviour"] = (
    df["Payment_Behaviour"]
    .astype(str)
    .str.replace("High_spent_Small_value_payments", "4")
)
df["Payment_Behaviour"] = (
    df["Payment_Behaviour"]
    .astype(str)
    .str.replace("High_spent_Medium_value_payments", "5")
)
df["Payment_Behaviour"] = (
    df["Payment_Behaviour"]
    .astype(str)
    .str.replace("High_spent_Large_value_payments", "6")
)

In [10]:
# Transform the object data the be float data type
col_int2 = [
    "Age",
    "Delay_from_due_date",
    "Num_of_Delayed_Payment",
    "Outstanding_Debt",
    "Total_EMI_per_month",
    "Monthly_Balance",
    "Payment_Behaviour",
    "Credit_History_Age",
    "Annual_Income",
]
for i in col_int2:
    df[i] = df[i].astype(float)

In [11]:
# Convert target variable to numeric
df["Credit_Score"] = df["Credit_Score"].str.replace("Good", "3", n=-1)
df["Credit_Score"] = df["Credit_Score"].str.replace("Standard", "2", n=-1)
df["Credit_Score"] = df["Credit_Score"].str.replace("Poor", "1", n=-1)
df["Credit_Score"] = df[["Credit_Score"]].apply(pd.to_numeric)

df["Payment_of_Min_Amount"] = df["Payment_of_Min_Amount"].str.replace("NM", "0")
df["Payment_of_Min_Amount"] = df["Payment_of_Min_Amount"].str.replace("Yes", "1")
df["Payment_of_Min_Amount"] = df["Payment_of_Min_Amount"].str.replace("No", "2")
df["Payment_of_Min_Amount"] = df[["Payment_of_Min_Amount"]].apply(pd.to_numeric)

In [12]:
df.dtypes

Age                       float64
Annual_Income             float64
Delay_from_due_date       float64
Num_of_Delayed_Payment    float64
Outstanding_Debt          float64
Credit_History_Age        float64
Payment_of_Min_Amount       int64
Total_EMI_per_month       float64
Payment_Behaviour         float64
Monthly_Balance           float64
Credit_Score                int64
dtype: object

In [13]:
# Split the dataset
X = df.loc[:, df.columns != "Credit_Score"]
Y = df["Credit_Score"]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [14]:
# Train the LightGBM model
lgbm = LGBMClassifier()
lgbm.fit(x_train, y_train)
y_pred = lgbm.predict(x_test)

In [15]:
# Print performance metrics
print("F1 score: %.3f" % f1_score(y_test, y_pred, average="weighted"))
print("Precision: %.3f" % precision_score(y_test, y_pred, average="weighted"))
print("Recall: %.3f" % recall_score(y_test, y_pred, average="weighted"))

F1 score: 0.681
Precision: 0.681
Recall: 0.681


During our model development, we gathered several performance metrics, including an F1 score of 0.810, a precision score of 0.818, and a recall score of 0.807. Subsequent monitoring can be evaluated similarly. For instance, if the F1 score falls below 0.75, it serves as an alert, prompting us to take immediate action to mitigate the issue.

# K-S Test

To demonstrate how the K-S Test works, I have selected a numeric feature `Credit_History_Age`. We will examine the sensitivity of this statistical test by creating three distinct datasets: one with 1000 samples, another with 5000 samples, and a third with approximately 64000 samples, which corresponds to the total size of the cleaned training samples. Each data point within the `Credit_History_Age` feature has been randomly picked from the training data and altered by a random floating-point number.

In [16]:
from scipy import stats

In [17]:
# Create new datasets with different no. of samples
original_df = x_train[["Credit_History_Age", "Payment_Behaviour"]].reset_index(
    drop=True
)
new_df = x_train[["Credit_History_Age", "Payment_Behaviour"]].reset_index(drop=True)
new_df1 = new_df.sample(n=1000).reset_index(drop=True)
new_df2 = new_df.sample(n=5000).reset_index(drop=True)
new_df3 = new_df.sample(n=len(x_train)).reset_index(drop=True)

In [18]:
# Prepare drifted data for numeric feature
def drift_numeric_col(df, numeric_col, drift_range):
    df[numeric_col] = df[numeric_col] + np.random.uniform(
        0, drift_range, size=(df.shape[0],)
    )

In [19]:
drift_numeric_col(new_df1, "Credit_History_Age", 2)
drift_numeric_col(new_df2, "Credit_History_Age", 2)
drift_numeric_col(new_df3, "Credit_History_Age", 2)

In [20]:
# K-S Test
def ks_test(original_df, new_df, numeric_col):
    test = stats.ks_2samp(original_df[numeric_col], new_df[numeric_col])
    print("Column : %s , p-value : %1.3f" % (numeric_col, test[1]))

In [21]:
# Conduct K-S Test for numeric feature
ks_test(original_df, new_df1, "Credit_History_Age")
ks_test(original_df, new_df2, "Credit_History_Age")
ks_test(original_df, new_df3, "Credit_History_Age")

Column : Credit_History_Age , p-value : 0.015
Column : Credit_History_Age , p-value : 0.000
Column : Credit_History_Age , p-value : 0.000


We obtained the following results:

- For the dataset with 1000 samples, the p-value of the K-S Test is 0.093.
- For the dataset with 5000 samples, the p-value of the K-S Test is 0.002.
- For the dataset with around 64000 samples, the p-value of the K-S Test is 0.000.

When examining the data drift scenario using 1000 samples, the p-value exceeded 0.05. Consequently, we can conclude that the two distributions remain similar. However, as the sample size increased to 5000 and beyond, the K-S Test exhibited exceptional performance, yielding p-values significantly lower than 0.05. This provides a clear indication of a data distribution drift, serving as a clear alert.

# PSI

In addition to utilizing the K-S Test, we will also leverage the PSI (Population Stability Index) to evaluate the numeric feature `Credit_History_Age` and assess the categorical feature `Payment_Behaviour`. To represent the drift effect, we have randomly replaced 80% of the data of the feature `Payment_Behaviour` with specific label values.

In [22]:
import random

In [23]:
# Prepare drifted data for categorical column
def drift_cat_col(df, cat_col, drift_ratio):
    no_of_drift = round(len(df) * drift_ratio)
    random_numbers = [random.randint(0, 1) for _ in range(no_of_drift)]
    indices = random.sample(range(len(df[cat_col])), no_of_drift)
    df.loc[indices, cat_col] = random_numbers

In [24]:
drift_cat_col(new_df1, "Payment_Behaviour", 0.8)
drift_cat_col(new_df2, "Payment_Behaviour", 0.8)
drift_cat_col(new_df3, "Payment_Behaviour", 0.8)

In [25]:
def psi(data_base, data_new, num_bins=10):
    # Sort the data
    data_base = sorted(data_base)
    data_new = sorted(data_new)

    # Prepare the bins
    min_val = min(data_base[0], data_new[0])
    max_val = max(data_base[-1], data_new[-1])
    bins = [min_val + (max_val - min_val) * (i) / num_bins for i in range(num_bins + 1)]
    bins[0] = min_val - 0.0001
    bins[-1] = max_val + 0.0001

    # Bucketize the baseline data and count the samples
    bins_base = pd.cut(data_base, bins=bins, labels=range(1, num_bins + 1))
    df_base = pd.DataFrame({"base": data_base, "bin": bins_base})
    grp_base = df_base.groupby("bin").count()
    grp_base["percent_base"] = grp_base["base"] / grp_base["base"].sum()

    # Bucketize the new data and count the samples
    bins_new = pd.cut(data_new, bins=bins, labels=range(1, num_bins + 1))
    df_new = pd.DataFrame({"new": data_new, "bin": bins_new})
    grp_new = df_new.groupby("bin").count()
    grp_new["percent_new"] = grp_new["new"] / grp_new["new"].sum()

    # Compare the bins
    psi_df = grp_base.join(grp_new, on="bin", how="inner")

    # Calculate the PSI
    psi_df["percent_base"] = psi_df["percent_base"].replace(0, 0.0001)
    psi_df["percent_new"] = psi_df["percent_new"].replace(0, 0.0001)
    psi_df["psi"] = (psi_df["percent_base"] - psi_df["percent_new"]) * np.log(
        psi_df["percent_base"] / psi_df["percent_new"]
    )

    # Return the total PSI value
    return np.sum(psi_df["psi"].values)

In [26]:
# Conduct K-S Test for numeric feature
psi(original_df["Credit_History_Age"], new_df1["Credit_History_Age"])

0.026888669992667112

In [27]:
psi(original_df["Credit_History_Age"], new_df2["Credit_History_Age"])

0.03748601241124246

In [28]:
psi(original_df["Credit_History_Age"], new_df3["Credit_History_Age"])

0.03182059320799732

- Numeric feature Credit_History_Age
With a sample size of 1000, the PSI value is 0.023.

With a sample size of 5000, the PSI value is 0.015.

With a sample size of approximately 64000, the PSI value is 0.021.

In [29]:
# Conduct K-S Test for categorical feature
psi(original_df["Payment_Behaviour"], new_df1["Payment_Behaviour"])

4.325712516729441

In [30]:
psi(original_df["Payment_Behaviour"], new_df2["Payment_Behaviour"])

4.343405836146468

In [31]:
psi(original_df["Payment_Behaviour"], new_df3["Payment_Behaviour"])

4.373471810217242

- Categorical feature Payment_Behaviour
With a sample size of 1000, the PSI value is 0.108.

With a sample size of 5000, the PSI value is 0.111.

With a sample size of approximately 64000, the PSI value is 0.112.

All the PSI values for the Credit_History_Age feature are significantly lower than 0.1, indicating no significant distribution change. By comparing these results with those obtained from the K-S Test, we observe that the K-S Test exhibits greater sensitivity in detecting distribution changes compared to PSI.

On the other hand, the PSI values for the Payment_Behaviour feature are around 0.11, signifying a moderate distribution change. Interestingly, the three PSI values remain relatively consistent, implying that PSI’s effectiveness is less dependent on sample sizes. Besides, PSI has the flexibility to monitor across various feature types, positioning it still a valuable approach for drift detection.

# ADWIN algorithm

Lastly, we test the power of ADWIN in detecting the change in the numeric feature Credit_History_Age. The [data stream](https://en.wikipedia.org/wiki/Streaming_data) consists of the training data, followed by the drifted data. We expect that the algorithm will promptly identify the drift shortly after examining the entirety of the original data.

To visually represent the situation, a scatter plot is created to showcase the final 500 points of the original data in blue, followed by the initial 500 points of the drifted data, depicted in green. The drifted data exhibits a slightly higher average value.

In [32]:
from skmultiflow.drift_detection import ADWIN

In [33]:
adwin = ADWIN()
data_stream = []
data_stream = np.concatenate(
    (original_df["Credit_History_Age"], new_df3["Credit_History_Age"])
)

In [34]:
# Add stream elements to ADWIN and verify if drift occurred
for i in range(len(data_stream)):
    adwin.add_element(data_stream[i])
    if adwin.detected_change():
        print('Change detected at index {}'.format(i))
    adwin.reset()

By continuously adding stream elements, ADWIN identifies the change at index 64457, which is the 637th data point within the drifted data. In comparison, the K-S Test and PSI necessitate a larger number of data points to conclude the presence of a drift confidently. The better performance of ADWIN is a testament to its capability to monitor diverse features at speed and ease.

# Wrapping it up

We’ve delved into the crucial concepts of data drift and model drift, which can lead to model decay in production. We can proactively monitor and detect drift conditions using model performance metrics, statistical tests, and adaptive windowing techniques. Unlike participating in a one-time Kaggle competition, building a production-ready ML system is an iterative journey. It requires a mindset shift towards integrating comprehensive model monitoring to ensure a robust and consistently high-performance serving space.